<a href="https://colab.research.google.com/github/Kaychuck/Predicting-Harmful-Algal-Blooms/blob/main/Miami%20River%20Predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Installing PySpark:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

In [2]:
# Installing PySpark:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [3]:
# Installing PySpark:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better
spark

In [10]:
df = spark.read.csv('/content/CREST Buoy 2 Miami River 030620 - 082420.csv', inferSchema=True,
                    header=True)
df

UnixTimestamp,Latitude (degrees),Longitude (degrees),Date,Time,Temp_C,spCond_uS_cm,Sal_psu,ODO_pctplus_pct,ODO_Concplus_mg_L,pH,Turbidity_NTU,Turbidity_FNU,Chlorophyll_ug_L,BGAPE_ug_L,BGAPE_RFU,FDOM_QSU,FDOM_RFU,Depth_meters,AbsSpeed_cm_s,Direction_Deg_M,DCS SP Std dev_cm_s,Signal Strength_dB,System Battery,System Temp,Internal RH
1.58351E12,25.7837,-80.21853,3/6/2020,16:16:03,25.438,31835.36,19.82,88.92,6.52,7.89,607.17,607.17,10.7,19.43,7.22,53.18,17.74,0.426,5.886021,253.3289,4.081941,-34.92703,13.61,32.08,58.82
1.58352E12,25.7837,-80.21853,3/6/2020,17:46:03,25.353,31640.56,19.69,89.87,6.6,7.89,306.89,306.89,11.71,18.32,6.82,53.42,17.82,0.431,7.723872,256.654,4.159851,-34.75467,13.18,31.52,59.8
1.58353E12,25.7837,-80.21853,3/6/2020,20:16:03,24.885,29172.17,18.01,83.68,6.26,7.86,527.79,527.79,11.04,17.66,6.57,63.07,21.04,0.453,7.825334,256.0207,3.400232,-38.3139,13.12,26.22,62.8
1.58353E12,25.7837,-80.21853,3/6/2020,20:46:03,24.781,28200.51,17.36,82.25,6.18,7.85,398.91,398.91,9.43,14.36,5.35,66.44,22.16,0.458,9.536228,259.6292,4.100551,-38.52298,13.11,25.67,62.83
1.58354E12,25.7837,-80.21853,3/7/2020,1:31:03,24.388,28719.96,17.71,77.07,5.82,7.84,583.73,583.73,6.68,10.94,4.09,67.17,22.4,0.479,1.185724,41.48075,3.57309,-38.53241,13.05,21.95,63.18
1.58355E12,25.7837,-80.21853,3/7/2020,2:01:03,24.246,28391.32,17.49,75.75,5.74,7.83,545.47,545.47,6.05,10.65,3.98,68.05,22.7,0.479,2.006407,83.56207,3.870536,-38.96541,13.04,21.43,63.36
1.58355E12,25.7837,-80.21853,3/7/2020,2:16:03,23.537,27475.2,16.88,76.08,5.86,7.83,550.53,550.53,6.19,10.46,3.91,69.98,23.34,0.477,4.746231,77.67785,3.623462,-38.90903,13.04,21.16,63.57
1.58355E12,25.7837,-80.21853,3/7/2020,2:46:03,23.289,27811.86,17.11,76.39,5.91,7.84,500.16,500.16,6.12,10.77,4.03,69.26,23.1,0.478,4.249239,61.54942,3.539034,-39.74473,13.01,20.76,63.77
1.58355E12,25.7837,-80.21853,3/7/2020,3:31:03,23.435,28707.74,17.72,77.2,5.93,7.84,483.74,483.74,5.76,9.52,3.56,67.78,22.61,0.476,1.682109,293.7963,3.965887,-37.41946,13.03,20.34,63.75
1.58356E12,25.7837,-80.21853,3/7/2020,4:31:03,23.416,28214.62,17.38,74.82,5.76,7.83,498.84,498.84,5.28,9.29,3.48,69.27,23.1,0.478,3.596662,249.8056,3.864616,-38.54076,13.01,19.56,63.95


In [11]:
from pyspark.sql import functions as F

renamed_df = df.select([F.col(col).alias(col.replace(' ', '_')) for col in df.columns])
renamed_df

UnixTimestamp,Latitude_(degrees),Longitude_(degrees),Date,Time,Temp_C,spCond_uS_cm,Sal_psu,ODO_pctplus_pct,ODO_Concplus_mg_L,pH,Turbidity_NTU,Turbidity_FNU,Chlorophyll_ug_L,BGAPE_ug_L,BGAPE_RFU,FDOM_QSU,FDOM_RFU,Depth_meters,AbsSpeed_cm_s,Direction_Deg_M,DCS_SP_Std_dev_cm_s,Signal_Strength_dB,System_Battery,System_Temp,Internal_RH
1.58351E12,25.7837,-80.21853,3/6/2020,16:16:03,25.438,31835.36,19.82,88.92,6.52,7.89,607.17,607.17,10.7,19.43,7.22,53.18,17.74,0.426,5.886021,253.3289,4.081941,-34.92703,13.61,32.08,58.82
1.58352E12,25.7837,-80.21853,3/6/2020,17:46:03,25.353,31640.56,19.69,89.87,6.6,7.89,306.89,306.89,11.71,18.32,6.82,53.42,17.82,0.431,7.723872,256.654,4.159851,-34.75467,13.18,31.52,59.8
1.58353E12,25.7837,-80.21853,3/6/2020,20:16:03,24.885,29172.17,18.01,83.68,6.26,7.86,527.79,527.79,11.04,17.66,6.57,63.07,21.04,0.453,7.825334,256.0207,3.400232,-38.3139,13.12,26.22,62.8
1.58353E12,25.7837,-80.21853,3/6/2020,20:46:03,24.781,28200.51,17.36,82.25,6.18,7.85,398.91,398.91,9.43,14.36,5.35,66.44,22.16,0.458,9.536228,259.6292,4.100551,-38.52298,13.11,25.67,62.83
1.58354E12,25.7837,-80.21853,3/7/2020,1:31:03,24.388,28719.96,17.71,77.07,5.82,7.84,583.73,583.73,6.68,10.94,4.09,67.17,22.4,0.479,1.185724,41.48075,3.57309,-38.53241,13.05,21.95,63.18
1.58355E12,25.7837,-80.21853,3/7/2020,2:01:03,24.246,28391.32,17.49,75.75,5.74,7.83,545.47,545.47,6.05,10.65,3.98,68.05,22.7,0.479,2.006407,83.56207,3.870536,-38.96541,13.04,21.43,63.36
1.58355E12,25.7837,-80.21853,3/7/2020,2:16:03,23.537,27475.2,16.88,76.08,5.86,7.83,550.53,550.53,6.19,10.46,3.91,69.98,23.34,0.477,4.746231,77.67785,3.623462,-38.90903,13.04,21.16,63.57
1.58355E12,25.7837,-80.21853,3/7/2020,2:46:03,23.289,27811.86,17.11,76.39,5.91,7.84,500.16,500.16,6.12,10.77,4.03,69.26,23.1,0.478,4.249239,61.54942,3.539034,-39.74473,13.01,20.76,63.77
1.58355E12,25.7837,-80.21853,3/7/2020,3:31:03,23.435,28707.74,17.72,77.2,5.93,7.84,483.74,483.74,5.76,9.52,3.56,67.78,22.61,0.476,1.682109,293.7963,3.965887,-37.41946,13.03,20.34,63.75
1.58356E12,25.7837,-80.21853,3/7/2020,4:31:03,23.416,28214.62,17.38,74.82,5.76,7.83,498.84,498.84,5.28,9.29,3.48,69.27,23.1,0.478,3.596662,249.8056,3.864616,-38.54076,13.01,19.56,63.95


In [12]:
renamed_df.createOrReplaceTempView('Data')

In [13]:
spark.sql('Select * from Data')

UnixTimestamp,Latitude_(degrees),Longitude_(degrees),Date,Time,Temp_C,spCond_uS_cm,Sal_psu,ODO_pctplus_pct,ODO_Concplus_mg_L,pH,Turbidity_NTU,Turbidity_FNU,Chlorophyll_ug_L,BGAPE_ug_L,BGAPE_RFU,FDOM_QSU,FDOM_RFU,Depth_meters,AbsSpeed_cm_s,Direction_Deg_M,DCS_SP_Std_dev_cm_s,Signal_Strength_dB,System_Battery,System_Temp,Internal_RH
1.58351E12,25.7837,-80.21853,3/6/2020,16:16:03,25.438,31835.36,19.82,88.92,6.52,7.89,607.17,607.17,10.7,19.43,7.22,53.18,17.74,0.426,5.886021,253.3289,4.081941,-34.92703,13.61,32.08,58.82
1.58352E12,25.7837,-80.21853,3/6/2020,17:46:03,25.353,31640.56,19.69,89.87,6.6,7.89,306.89,306.89,11.71,18.32,6.82,53.42,17.82,0.431,7.723872,256.654,4.159851,-34.75467,13.18,31.52,59.8
1.58353E12,25.7837,-80.21853,3/6/2020,20:16:03,24.885,29172.17,18.01,83.68,6.26,7.86,527.79,527.79,11.04,17.66,6.57,63.07,21.04,0.453,7.825334,256.0207,3.400232,-38.3139,13.12,26.22,62.8
1.58353E12,25.7837,-80.21853,3/6/2020,20:46:03,24.781,28200.51,17.36,82.25,6.18,7.85,398.91,398.91,9.43,14.36,5.35,66.44,22.16,0.458,9.536228,259.6292,4.100551,-38.52298,13.11,25.67,62.83
1.58354E12,25.7837,-80.21853,3/7/2020,1:31:03,24.388,28719.96,17.71,77.07,5.82,7.84,583.73,583.73,6.68,10.94,4.09,67.17,22.4,0.479,1.185724,41.48075,3.57309,-38.53241,13.05,21.95,63.18
1.58355E12,25.7837,-80.21853,3/7/2020,2:01:03,24.246,28391.32,17.49,75.75,5.74,7.83,545.47,545.47,6.05,10.65,3.98,68.05,22.7,0.479,2.006407,83.56207,3.870536,-38.96541,13.04,21.43,63.36
1.58355E12,25.7837,-80.21853,3/7/2020,2:16:03,23.537,27475.2,16.88,76.08,5.86,7.83,550.53,550.53,6.19,10.46,3.91,69.98,23.34,0.477,4.746231,77.67785,3.623462,-38.90903,13.04,21.16,63.57
1.58355E12,25.7837,-80.21853,3/7/2020,2:46:03,23.289,27811.86,17.11,76.39,5.91,7.84,500.16,500.16,6.12,10.77,4.03,69.26,23.1,0.478,4.249239,61.54942,3.539034,-39.74473,13.01,20.76,63.77
1.58355E12,25.7837,-80.21853,3/7/2020,3:31:03,23.435,28707.74,17.72,77.2,5.93,7.84,483.74,483.74,5.76,9.52,3.56,67.78,22.61,0.476,1.682109,293.7963,3.965887,-37.41946,13.03,20.34,63.75
1.58356E12,25.7837,-80.21853,3/7/2020,4:31:03,23.416,28214.62,17.38,74.82,5.76,7.83,498.84,498.84,5.28,9.29,3.48,69.27,23.1,0.478,3.596662,249.8056,3.864616,-38.54076,13.01,19.56,63.95


In [ ]:
spark.sql('select count(State_UTs) from Data')